Para esta iteracion, calcularemos usaremos feature engineering para mejorar las metricas de nuestro modelo.

Buildeamos codigo C++
Mover el build generado en la carpeta experimentacion a la carpeta requerida(en este caso experimentacion_precio)

In [ ]:
!cd ../../ && mkdir build
!cd ../../build/ && rm -rf *
!cd ../../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../../build && make install

Importamos dependencias

In [1]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame
import ModeloM2
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score, mean_absolute_error
from pathlib import Path

Seteamos nuestro dataset

In [2]:
df = pd.read_csv('../../data/train.csv')

Como en los ultimos experimentos, testearemos nuestro modelo con feature engineering con distintas combinaciones de las dos columnas que agragamos la modelo.

In [4]:
KFOLD_K = 5
kf = KFold(n_splits=KFOLD_K, shuffle=True)

result = {}
for name, tuple in [('piscina efectiva' , (True, False, False)), ('garage efectiva', (False, True, False)), ('espacioso', (False, False, True))]:
    rms = []
    rmsle = []
    mae = []
    r2 = []
    df = df[df['metroscubiertos'].notna()]
    df.reset_index(drop=True, inplace=True)
    for train_index, test_index in kf.split(df):
        df_train, df_test = df.loc[train_index], df.loc[test_index]
        
        mp = ModeloM2.ModeloV2FeatEng(df_train, tuple)
        df_predicted = mp.run(df_test, ['tipodepropiedad','garages'])
    
        
        df_predicted['metroscubiertos'] = df_predicted['metroscubiertos'].copy().apply(lambda x: x if x > 0 else 0) 
        rms.append(mean_squared_error(df_test["metroscubiertos"], df_predicted["metroscubiertos"], squared=False))
        rmsle.append(mean_squared_log_error(df_test["metroscubiertos"],  df_predicted['metroscubiertos']))
        r2.append(r2_score(df_test["metroscubiertos"], df_predicted["metroscubiertos"]))
        mae.append(mean_absolute_error(df_test["metroscubiertos"], df_predicted["metroscubiertos"]))

    rms_result = np.sum(rms) / KFOLD_K 
    rmsle_result = np.sum(rmsle) / KFOLD_K 
    r2_result = np.sum(r2) / KFOLD_K 
    mae_result = np.sum(mae) / KFOLD_K

    result[name] = [rms_result, rmsle_result, r2_result, mae_result]


out = pd.DataFrame.from_dict(result, orient='index', columns=['RMSE', 'RMSLE', 'R^2', 'MAE']).to_latex(column_format='p{6cm}|r|r|r|r')
print(out)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().